In [6]:
import numpy as np
import math
L,d_k,d_v = 4,8,8,
#L = 길이 length of sequence
#d = dimension
q = np.random.randn(L,d_k)
k = np.random.randn(L,d_k)
v = np.random.randn(L,d_v)



In [18]:
q.shape


(4, 8)

In [19]:
#q = np.random.randn(L,d_k); q.T.shape;

(8, 4)

In [7]:
np.matmul(q,k.T) #q*k^^T
#.T하는이유 (4,8)*(4,8) shape multiplication은 안됨으로 (4,8)*(8,4)로 만드는거임

array([[-0.50107988, -2.03895317, -1.92675236,  1.26607212],
       [-0.06259682, -2.85755388,  1.94029806, -0.87769011],
       [ 1.89836605, -1.10229663,  6.66723649, -5.2048896 ],
       [ 0.50723831, -4.38114975,  4.80747731, -1.3636343 ]])

In [8]:
scaled = np.matmul(q,k.T)/ math.sqrt(d_k)

# 이게 뭐냐 self attention 의 softmax version
#효과
q.var(), k.var(), np.matmul(q,k.T).var(), scaled.var()

(0.8958212662555859, 1.1919224751339506, 8.910075266419014, 1.1137594083023767)

In [13]:
mask = np.tril(np.ones((L,L))) # value 1 shape L,L 을 대각선으로 값을지움
mask
mask[mask==0] =-np.infty
mask[mask==1] = 0

In [14]:
scaled + mask # mask된 matrix

array([[-0.17715849,        -inf,        -inf,        -inf],
       [-0.02213132, -1.01029786,        -inf,        -inf],
       [ 0.67117375, -0.38972071,  2.35722407,        -inf],
       [ 0.17933582, -1.54897035,  1.6996999 , -0.48211753]])

In [27]:
def softmax(x):
  return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T
  #axis -1 은 열에따라 합치라는 뜻 shape 3,3에 keepdim 썻으면 3,1로

In [29]:
#array = np.array([[1,2,3],[4,5,6],[7,8,9]])
#sum = np.sum(array,axis=0)
#sum

In [32]:
attention = softmax(scaled + mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.72872563, 0.27127437, 0.        , 0.        ],
       [0.14827415, 0.05132451, 0.80040133, 0.        ],
       [0.15955132, 0.02833391, 0.72977021, 0.08234455]])

In [35]:
out = np.matmul(attention,v)
out

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [51]:
sequence_length = 4
batch_size = 1
input_dim = 512
d_model = 512
#hyperparameter 설정

# input 생성
x=torch.randn((batch_size, sequence_length, input_dim))
x

tensor([[[-0.3934,  0.1299,  0.9010,  ..., -0.6986, -1.5063,  0.8542],
         [ 1.5634,  1.9696,  0.5767,  ..., -0.8874,  0.1565, -1.0368],
         [ 0.1959,  0.4534, -1.2755,  ..., -0.8263,  0.1567, -1.2787],
         [ 0.3998,  0.1143,  0.0947,  ..., -0.7793,  0.5126,  0.6687]]])

In [52]:
qkv_layer = nn.Linear(input_dim,3*d_model)


In [53]:
qkv = qkv_layer(x)


In [56]:
qkv.shape


torch.Size([1, 4, 1536])

In [64]:
#number of heads"가 8이면 입력 시퀀스는 8개의 부분으로 나눠짐.
#각 부분은 각각의 attention head에 의해 독립적으로 처리됨.
#각 head는 다양한 특징에 주의를 기울이도록 훈련됨
# 보통 8아니면 16
nums_heads = 8
head_dim = d_model // nums_heads
qkv = qkv.reshape(batch_size,sequence_length,nums_head,3*head_dim)
qkv.shape

torch.Size([1, 4, 8, 192])

In [65]:
#병렬 처리를 쉽게 하기위해 permute
qkv = qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [66]:
q, k, v = qkv.chunk(3,dim=-1)
#4번쨰 dimension으로 3개로 나눔
q.shape

torch.Size([1, 8, 4, 64])

In [77]:
#softmax 적용 이번엔 4차원임으로 transpose사용
d_k = q.size()[-1]
#head_dim으로 설정
scaled = torch.matmul(q,k.transpose(-2,-1))/math.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [86]:
k.T.shape #1,8,4,64 -> 64,4,8,1
mask = torch.full(scaled.size(),float('-inf'))
mask # 1,8,4,4 shape로 inf 만 채워서 생성
mask = torch.triu(mask,diagonal=1)
mask

tensor([[[[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0.,

In [87]:
(scaled + mask)[0][0]

tensor([[ 0.0447,    -inf,    -inf,    -inf],
        [-0.5308, -0.1147,    -inf,    -inf],
        [ 0.3128,  0.6896, -0.6147,    -inf],
        [-0.2300, -0.5748, -0.0061, -0.0013]], grad_fn=<SelectBackward0>)

In [91]:
scaled = scaled + mask
attention = F.softmax(scaled,dim = -1)
# -1 하면 위의 scaled[0][0][0][2] 에서 softmax가 진행된다
np.exp(-0.538) / (np.exp(-0.538)+ np.exp(-0.1147))

0.39572735940832304

In [89]:
attention[0][0] # 캬~

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.3975, 0.6025, 0.0000, 0.0000],
        [0.3505, 0.5109, 0.1386, 0.0000],
        [0.2372, 0.1680, 0.2967, 0.2981]], grad_fn=<SelectBackward0>)

In [95]:
values = torch.matmul(attention,v)

In [96]:
values.size()

torch.Size([1, 8, 4, 64])

In [100]:
#multi head attention 종료!! linear layer에 넣으면
values =values.reshape(batch_size,sequence_length,nums_heads*head_dim)

In [ ]:
values

In [98]:
linear_layer = nn.Linear(d_model,d_model)

In [101]:
out =  linear_layer(values) #input 보다 문맥이해도 증가